In [1]:
from ast import literal_eval
import pandas as pd
import numpy as np
df = pd.read_csv('/Users/kexinhuang/Downloads/TDC Leaderboard Submission Form 5.csv')

In [2]:
df = df.rename(columns = {
    df.columns.values[0]: 'Time',
    df.columns.values[1]: 'Name',
    df.columns.values[2]: 'Email',
    df.columns.values[3]: 'Model',
    df.columns.values[4]: 'Paper_URL',
    df.columns.values[5]: 'Code_URL',
    df.columns.values[6]: 'Hardware',
    df.columns.values[7]: '#Params',
    df.columns.values[8]: 'Group',
    df.columns.values[9]: 'Text_Result',
    df.columns.values[10]: 'PKL_Result',    
})
df = df.drop(df.columns.values[11], axis = 1)

In [3]:
def return_text(x):
    try:
        y = literal_eval(x)
    except:
        y = x
    return y
df['Result'] = df.Text_Result.apply(lambda x: return_text(x))

In [4]:
metrics_direction = {
    'mae': 'desc',
    'roc-auc': 'asc', 
    'pr-auc': 'asc',
    'pcc': 'asc'
}

In [5]:
def retrieve_mean(x, categories):
    l = []
    for i in categories:
        l.append(x[i][0])
    return np.array(l)

## ADMET Group

In [39]:
df_admet = df[df.Group == 'ADMET Group']

In [40]:
admet_metrics = {'caco2_wang': 'mae',
            'hia_hou': 'roc-auc',
            'pgp_broccatelli': 'roc-auc', 
            'bioavailability_ma': 'roc-auc',
            'lipophilicity_astrazeneca': 'mae',
            'solubility_aqsoldb': 'mae',
            'bbb_martins': 'roc-auc',
            'ppbr_az': 'mae',
            'vdss_lombardo': 'pcc',
            'cyp2c9_veith': 'pr-auc',
            'cyp2d6_veith': 'pr-auc',
            'cyp3a4_veith': 'pr-auc',
            'cyp2c9_substrate_carbonmangels': 'pr-auc',
            'cyp3a4_substrate_carbonmangels': 'roc-auc',
            'cyp2d6_substrate_carbonmangels': 'pr-auc',
            'half_life_obach': 'pcc',
            'clearance_hepatocyte_az': 'pcc',
            'clearance_microsome_az': 'pcc',
            'ld50_zhu': 'mae',
            'herg': 'roc-auc',
            'ames': 'roc-auc',
            'dili': 'roc-auc'
            }

In [41]:
ADMET = {
    'Absorption': ['caco2_wang', 'hia_hou', 'pgp_broccatelli', 'bioavailability_ma', 'lipophilicity_astrazeneca', 'solubility_aqsoldb'],
    'Distribution': ['bbb_martins', 'ppbr_az', 'vdss_lombardo'],
    'Metabolism': ['cyp2c9_veith', 'cyp2d6_veith', 'cyp3a4_veith', 'cyp2c9_substrate_carbonmangels', 'cyp2d6_substrate_carbonmangels', 'cyp3a4_substrate_carbonmangels'],
    'Excretion': ['half_life_obach', 'clearance_hepatocyte_az', 'clearance_microsome_az'],
    'Toxicity': ['ld50_zhu', 'herg', 'ames', 'dili']
}

In [43]:
def get_entries(bm):
    results = df_admet.Result.apply(lambda x: x[bm][0]).values
    print(results)
    
    '''
    rank_dict = dict(zip(range(results.shape[0]), list([0] * results.shape[0])))
    
    for idx, i in enumerate(range(results.T.shape[0])):
        # for every benchmark
        sort_idx = np.argsort(results.T[i], axis = 0)
        if metrics_direction[admet_metrics[ADMET[category][idx]]] == 'desc':
            sort_idx = np.argsort(results.T[i], axis = 0)
        else:
            sort_idx = np.argsort(results.T[i], axis = 0)[::-1]
        
        for pos, val in enumerate(sort_idx):
            rank_dict[val] = rank_dict[val] + pos
    
    rank = [] # [rank for 1, rank for 2, ...]
    for i in range(results.shape[0]):
        rank.append(rank_dict[i])
        
    out = np.argsort(rank)
    
    idx2rank = dict(zip(out, list(range(len(out)))))
    rank2idx = dict(zip(list(range(len(out))), out))
    '''
    out = np.argsort(results)
    idx2rank = dict(zip(out, list(range(len(out)))))
    rank2idx = dict(zip(list(range(len(out))), out))
    
    print('---- ' + bm + ' ------')
    print('---- ' + admet_metrics[bm] + ' ------')
    entries = []
    for submit_idx in range(len(df_admet)):
        temp = df_admet.iloc[submit_idx]
        html_code = '<tr> \n'
        html_code = html_code + '  <td> ' + str(idx2rank[submit_idx] + 1) + ' </td> \n'        
        html_code = html_code + '  <td> ' + temp.Model + ' </td> \n'
        html_code = html_code + '  <td><a href="mailto:' + temp.Email + '">' + temp.Name + '</a></td> \n'
        html_code = html_code + '  <td><a href="' + temp.Code_URL + '"> GitHub</a>, <a href="' + temp.Paper_URL + '">Paper </a></td> \n'
        value = temp['#Params']
        html_code = html_code + '  <td> ' + f'{value:,}' + ' </td> \n'
        html_code = html_code + '  <td>' + format(temp.Result[bm][0], '.3f') + ' <span>&#177;</span> ' + format(temp.Result[bm][1], '.3f') +' </td> \n'
        html_code += '</tr>'

        entries.append(html_code)
    
    for i in range(len(out)):
        print(entries[rank2idx[i]])

In [44]:
for i, j in ADMET.items():
    for x in j:
        get_entries(x)

[0.446 0.393 0.908]
---- caco2_wang ------
---- mae ------
<tr> 
  <td> 1 </td> 
  <td> RDKit2D + MLP (DeepPurpose) </td> 
  <td><a href="mailto:kexinhuang@hsph.harvard.edu">Kexin Huang</a></td> 
  <td><a href="https://github.com/mims-harvard/TDC/tree/master/examples/single_pred/admet"> GitHub</a>, <a href="https://doi.org/10.1093/bioinformatics/btaa1005">Paper </a></td> 
  <td> 633,409 </td> 
  <td>0.393 <span>&#177;</span> 0.024 </td> 
</tr>
<tr> 
  <td> 2 </td> 
  <td> CNN (DeepPurpose) </td> 
  <td><a href="mailto:kexinhuang@hsph.harvard.edu">Kexin Huang</a></td> 
  <td><a href="https://github.com/mims-harvard/TDC/tree/master/examples/single_pred/admet"> GitHub</a>, <a href="https://doi.org/10.1093/bioinformatics/btaa1005">Paper </a></td> 
  <td> 226,625 </td> 
  <td>0.446 <span>&#177;</span> 0.036 </td> 
</tr>
<tr> 
  <td> 3 </td> 
  <td> Morgan + MLP (DeepPurpose) </td> 
  <td><a href="mailto:kexinhuang@hsph.harvard.edu">Kexin Huang</a></td> 
  <td><a href="https://github.com/mim

## Docking Group

In [6]:
df_docking = df[df.Group == 'Docking Group']

In [9]:
from Google import Create_Service

CLIENT_SECRET_FILE = 'client_secret_file.json'
API_NAME = 'drive'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/drive']

service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)

client_secret_file.json-drive-v3-(['https://www.googleapis.com/auth/drive'],)
['https://www.googleapis.com/auth/drive']
drive service created successfully


In [10]:
import os, io
from googleapiclient.http import MediaIoBaseDownload

file_ids = [i.split('id=')[1] for i in df_docking.PKL_Result.values]
file_names = [i + '.pkl' for i in df_docking.Model.values]

for file_id, file_name in zip(file_ids, file_names):
    request = service.files().get_media(fileId = file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fd = fh, request = request)
    
    done = False
    while not done:
        status, done = downloader.next_chunk()
        print('Download progress {0}'.format(status.progress() * 100))

    fh.seek(0)

    with open(os.path.join('./docking_result', file_name), 'wb') as f:
        f.write(fh.read())
        f.close()

Download progress 100.0
Download progress 100.0
Download progress 100.0


In [11]:
import pickle
def get_result(x):
    with open(os.path.join('./docking_result', x + '.pkl'), 'rb') as f:      
        x = pickle.load(f)
    return x
              
df_docking['Result'] = df_docking.Model.apply(lambda x: get_result(x))

/Users/kexinhuang/opt/anaconda3/envs/DeepPurpose/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [39]:
## generate molecule images
from rdkit import Chem
from rdkit.Chem import Draw

for submit_idx in range(len(df_docking)):
    temp = df_docking.iloc[submit_idx]

    for target in temp.Result.keys():
        for max_calls in temp.Result[target].keys():
            mols = [Chem.MolFromSmiles(i) for i in temp.Result[target][max_calls]['top smiles']]
            img = Draw.MolsToGridImage(mols, molsPerRow=6, subImgSize=(200, 200), maxMols=300)
            file_path = 'benchmark/docking_group/smiles/' + target.lower() + '_img/' + temp.Model + '_' + str(max_calls) + '.png'
            img.save(file_path)

In [69]:
## best in data
pred = dict(pd.read_csv('../tdc_leaderboard/docking_zinc_drd3_top100', sep = '\t', header = None).values)

mols = [Chem.MolFromSmiles(i) for i in list(pred.keys())]
img = Draw.MolsToGridImage(mols, molsPerRow=6, subImgSize=(200, 200), maxMols=300)
file_path = 'benchmark/docking_group/smiles/' + target.lower() + '_img/best_in_data.png'
img.save(file_path)

In [46]:
## generate html page for that image
html_code = ''
all_targets = list(temp.Result.keys())

for target in all_targets:    
    for submit_idx in range(len(df_docking)):
        temp = df_docking.iloc[submit_idx]
        html_code += '## ' + temp.Model + ' \n'
        for max_calls in temp.Result[target].keys():
            html_code += '### ' + str(max_calls) + ' Maximum Calls \n'
            html_code += "\n"
            html_code += "<img src='/benchmark/docking_group/smiles/" + target.lower() + "_img/" + temp.Model + "_" + str(max_calls) + ".png'> \n"
            html_code += "\n"
            html_code += "<b> <i class='fas fa-long-arrow-alt-up'></i> <a href='/benchmark/docking_group/smiles/" + target.lower() + "_smiles'>Go Back To Table of Contents</a></b> \n"
            html_code += "\n"
            html_code += "<b> <i class='fas fa-long-arrow-alt-left'></i> <a href='/benchmark/docking_group/"+ target.lower() + "'> Go Back To Leaderboard</a> </b>"
            html_code += "\n"
            html_code += "\n"

In [47]:
print(html_code)

## GCPN 
### 100 Maximum Calls 

<img src='/benchmark/docking_group/smiles/drd3_img/GCPN_100.png'> 

<b> <i class='fas fa-long-arrow-alt-up'></i> <a href='/benchmark/docking_group/smiles/drd3_smiles'>Go Back To Table of Contents</a></b> 

<b> <i class='fas fa-long-arrow-alt-left'></i> <a href='/benchmark/docking_group/drd3'> Go Back To Leaderboard</a> </b>

### 500 Maximum Calls 

<img src='/benchmark/docking_group/smiles/drd3_img/GCPN_500.png'> 

<b> <i class='fas fa-long-arrow-alt-up'></i> <a href='/benchmark/docking_group/smiles/drd3_smiles'>Go Back To Table of Contents</a></b> 

<b> <i class='fas fa-long-arrow-alt-left'></i> <a href='/benchmark/docking_group/drd3'> Go Back To Leaderboard</a> </b>

### 1000 Maximum Calls 

<img src='/benchmark/docking_group/smiles/drd3_img/GCPN_1000.png'> 

<b> <i class='fas fa-long-arrow-alt-up'></i> <a href='/benchmark/docking_group/smiles/drd3_smiles'>Go Back To Table of Contents</a></b> 

<b> <i class='fas fa-long-arrow-alt-left'></i> <a href='/

In [63]:
def get_entries(bm, max_calls):
    results = df_docking.Result.apply(lambda x: x[bm][max_calls]['top100'][0]).values
    
    out = np.argsort(results)
    idx2rank = dict(zip(out, list(range(len(out)))))
    rank2idx = dict(zip(list(range(len(out))), out))
    
    print('---- ' + bm + ' ------')
    entries = []
    for submit_idx in range(len(df_docking)):
        temp = df_docking.iloc[submit_idx]
        html_code = '<tr> \n'
        html_code = html_code + '  <td> ' + str(idx2rank[submit_idx] + 1) + ' </td> \n'        
        html_code = html_code + '  <td> ' + temp.Model + ' </td> \n'
        html_code = html_code + '  <td><a href="mailto:' + temp.Email + '">' + temp.Name + '</a></td> \n'
        html_code = html_code + '  <td><a href="' + temp.Code_URL + '"> GitHub</a>, <a href="' + temp.Paper_URL + '">Paper </a></td> \n'
        value = temp['#Params']
        value = int(str(value).replace(',', ''))
        html_code = html_code + '  <td> ' + f'{value:,}' + ' </td> \n'
        for metric in ['top100', 'top10', 'top1', 'diversity', 'novelty', '%pass', 'top1_%pass', 'm1']:
            html_code = html_code + '  <td>' + format(temp.Result[bm][max_calls][metric][0], '.3f') + ' <span>&#177;</span> ' + format(temp.Result[bm][max_calls][metric][1], '.3f') +' </td> \n'
        
        html_code += "  <td> <a href='/benchmark/docking_group/smiles/" + bm.lower() + "_smiles'> Link </a> </td> \n"
        html_code += '</tr>'

        entries.append(html_code)
    
    for i in range(len(out)):
        print(entries[rank2idx[i]])

In [68]:
get_entries('DRD3', 100)

---- DRD3 ------
<tr> 
  <td> 1 </td> 
  <td> SMILES-LSTM </td> 
  <td><a href="mailto:tfu42@gatech.edu">Tianfan Fu</a></td> 
  <td><a href="https://github.com/futianfan/guacamol_tdc"> GitHub</a>, <a href="https://pubs.acs.org/doi/10.1021/acscentsci.7b00512">Paper </a></td> 
  <td> 3,149,000 </td> 
  <td>-7.594 <span>&#177;</span> 0.182 </td> 
  <td>-10.033 <span>&#177;</span> 0.186 </td> 
  <td>-11.133 <span>&#177;</span> 0.634 </td> 
  <td>0.884 <span>&#177;</span> 0.002 </td> 
  <td>1.000 <span>&#177;</span> 0.000 </td> 
  <td>0.763 <span>&#177;</span> 0.019 </td> 
  <td>-1.100 <span>&#177;</span> 1.417 </td> 
  <td>5.219 <span>&#177;</span> 0.247 </td> 
  <td> <a href='/benchmark/docking_group/smiles/drd3_smiles'> Link </a> </td> 
</tr>
<tr> 
  <td> 2 </td> 
  <td> Graph-GA </td> 
  <td><a href="mailto:tfu42@gatech.edu">Tianfan Fu</a></td> 
  <td><a href="https://github.com/futianfan/guacamol_tdc"> GitHub</a>, <a href="https://pubs.rsc.org/en/content/articlelanding/2019/sc/c8sc0537